In [ ]:
#!pip install  absl-py scann tensorflow-datasets google-cloud-bigquery tensorflow-recommenders google-cloud-aiplatform tensorflow-io --user

# Two-Tower Recommendation Example
# Import libs

In [ ]:
# !export TF_GPU_THREAD_MODE=gpu_private

In [1]:
import os
import time
import json

from absl import app
from absl import flags
from absl import logging


import tensorflow_recommenders as tfrs
import numpy as np
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
 
    
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes


import tensorflow as tf


from google.cloud import aiplatform as vertex_ai
from google.cloud import bigquery
from google.cloud import storage

## Set Vars

In [22]:
PREFIX = 'two-tower'
#PREFIX = 'css_retail'
DISPLAY_NAME = f'{PREFIX}-tensorboard'
PROJECT= 'vertex-stuff'
REGION='us-central1'

STAGING_BUCKET = """gs://{}_vertex_training""".format(PROJECT) #lowes-reccomendation-tensorboard-logs-us-central1 - this 
#TENSORBOARD = 'projects/258043323883/locations/us-central1/tensorboards/4236655796332527616' #note really can only get this after gcloud beta ai tensorboards create...
#VERTEX_SA = 'vertex-tb@lowes-reccomendation.iam.gserviceaccount.com'

FLAGS = flags.FLAGS
LR = 0.0002 #flags.DEFINE_float("LR", 0.001, "Learning Rate")
EMBEDDING_DIM = 64 #flags.DEFINE_integer("EMBEDDING_DIM", 15, "Embedding dimension")
MAX_TOKENS = 300_000 #flags.DEFINE_integer("MAX_TOKENS", 15, "Max embeddings for query and last_n products")
NUM_EPOCHS = 100 #flags.DEFINE_integer("NUM_EPOCHS", 29, "Number of epochs")
MODEL_DIR = 'model-dirs' #flags.DEFINE_string("MODEL_DIR", 'model-dirs-lowes', "GCS Bucket to store the model artifact")
DROPOUT = False #flags.DEFINE_bool("DROPOUT", False, "Use Dropout - T/F bool type")
DROPOUT_RATE = None #flags.DEFINE_float("DROPOUT_RATE", -1.4, "Dropout rate only works with DROPOUT=True")
#flags.DEFINE_integer("N_PRODUCTS", 19999, "number of products considered for embedding")
BATCH_SIZE = 256 #flags.DEFINE_integer("BATCH_SIZE", 1023, "batch size")
ARCH = '[1000,500,100]' #flags.DEFINE_string("ARCH", '[128,64]', "deep architecture, expressed as a list of ints in string format - will be parsed into list")
SEED = 8947 #flags.DEFINE_integer("SEED", 41781896, "random seed")
#flags.DEFINE_string("TF_RECORDS_DIR", "gs://tfrs-central-a", "source data in tfrecord format gcs location")

#TODO FLAG
DATASET = 'css_retail'
PROJECT_ID = 'vertex-stuff'
JOB_DIR = 'gs://jsw-bucket/'
N_BINS = 10


# initialize vertex sdk
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)

## Shape data for training in BQ

In [23]:
# client = bigquery.Client()
# product_catalog_sql = """
# CREATE OR REPLACE TABLE `css_retail.product_train` AS 
# WITH inner_q AS (
#     SELECT
#         SAFE_CAST(rad.id AS STRING) as productId,
#         title,
#         description,
#         product_metadata.exact_price.original_price AS price,
#         ARRAY_TO_STRING(cats.categories, ' ') AS categories
#     FROM `css_retail.recommendation_ai_data` AS rad,
#     UNNEST(category_hierarchies) AS cats
# ) 
# 	SELECT DISTINCT
#     productId,
#     title,
#     description,
#     price,
#     categories
# FROM inner_q
# GROUP BY productId, title, description, price, categories
# """
# ################### LIMIT FOR DEV ##############
# _ = client.query(product_catalog_sql)
# _.result()

In [24]:

import tensorflow as tf
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession


def read_dataset_prod(client, row_restriction, batch_size=1024):
    TABLE_ID = "product_train"
    COL_NAMES = ['productId', 'title', 'description', 'price', 'categories']
    COL_TYPES = [dtypes.string, dtypes.string, dtypes.string, dtypes.float64, dtypes.string]
    bqsession = client.read_session(
        "projects/" + PROJECT_ID,
        PROJECT_ID, TABLE_ID, DATASET,
        COL_NAMES, COL_TYPES,
        requested_streams=2,
        row_restriction=row_restriction)
    dataset = bqsession.parallel_read_rows()
    return dataset.prefetch(1).shuffle(batch_size*10).batch(batch_size)

def read_dataset(client, row_restriction, batch_size=1024):
    TABLE_ID = "train"
    COL_NAMES = ['userId', 'age', 'gender', 'latitude', 'longitude', 'zip', 'traffic_source',  'customer_lifetime_days',
                'productId', 'quantity', 'eventTime', 'hour', 'day', 'month', 'dow', 'title', 'description', 'price', 'categories']
    COL_TYPES = [dtypes.string, dtypes.float64, dtypes.string, dtypes.float64, dtypes.float64, dtypes.string, dtypes.string, dtypes.float64, dtypes.string, 
                 dtypes.int64, dtypes.int64, dtypes.string, dtypes.string, dtypes.string, dtypes.string, dtypes.string, dtypes.string, dtypes.float64,
                dtypes.string]
    bqsession = client.read_session(
        "projects/" + PROJECT_ID,
        PROJECT_ID, TABLE_ID, DATASET,
        COL_NAMES, COL_TYPES,
        requested_streams=2,
        row_restriction=row_restriction)
    dataset = bqsession.parallel_read_rows()
    return dataset.prefetch(1).shuffle(batch_size*10).batch(batch_size)

client = BigQueryClient()

In [25]:
# def read_bigquery(table_name, schema, unused, row_restriction=None):
#     tensorflow_io_bigquery_client = BigQueryClient()
#     read_session = tensorflow_io_bigquery_client.read_session(
#         "projects/" + PROJECT_ID,
#         PROJECT_ID, table_name, DATASET,
#     list(field.name for field in schema 
#         if not field.name in unused),
#     list(dtypes.double if field.field_type in ['FLOAT64']
#         else dtypes.int64 if field.field_type in ['INT64', 'INT32']
#          else dtypes.string for field in schema
#         if not field.name in unused),
#       requested_streams=10,
#     row_restriction=row_restriction)
    
#     dataset = read_session.parallel_read_rows()
#     # parsed_data = dataset.map(lambda x: x)
#     return dataset

## Product Model Class

In [26]:
class ProductModel(tf.keras.Model):
    def __init__(self, layer_sizes, adapt_data):
        super().__init__()
        
        #preprocess stuff
        logging.info('Preprocessing Running - product model')
        self.sku_count = len(np.unique(
            np.concatenate(
                list(
                    adapt_data.map(lambda x: x["productId"])
                )
            )
        ))
        #categorical: sku
        self.sku_vectorizor = tf.keras.layers.TextVectorization(max_tokens=self.sku_count,  name='sku_vec', input_shape=())
        self.title_vectorizor = tf.keras.layers.TextVectorization(max_tokens=self.sku_count,  name='title_vec', ngrams=2, input_shape=())
        self.description_vectorizor = tf.keras.layers.TextVectorization(max_tokens=self.sku_count, name='desc_vec', input_shape=())
        self.category_vectorizor = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS, name='cat_vec', ngrams=2, input_shape=())

        # self.price_normalization = tf.keras.layers.Normalization(axis=None)

        logging.info('Lookups Complete - product model')

        #adapt stuff
        logging.info('Adapts Running - product model')
        self.category_vectorizor.adapt(adapt_data.map(lambda x: x['categories']))
        self.title_vectorizor.adapt(adapt_data.map(lambda x: x['title']))
        self.description_vectorizor.adapt(adapt_data.map(lambda x: x['description']))
        self.sku_vectorizor.adapt(adapt_data.map(lambda x: x['productId']))
        # self.price_normalization.adapt(adapt_data.map(lambda x: x['price']))
        
        logging.info('Starting Emb Layers - product model')
        #embed stuff
        self.sku_embedding = tf.keras.Sequential(
            [
                self.sku_vectorizor,
                tf.keras.layers.Embedding(
                    self.sku_count+1,
                    EMBEDDING_DIM,
                    mask_zero=True,
                    name="sku_emb",
                    # input_shape=(self.sku_count,),
                ),
                tf.keras.layers.GlobalAveragePooling1D(
                    name="sku_flat",
                    # input_shape=(self.sku_count+1,EMBEDDING_DIM,)
                )
             ],
             name="sku_embedding")
        
        self.title_embedding = tf.keras.Sequential(
            [
                self.title_vectorizor,
                tf.keras.layers.Embedding(
                    self.sku_count+1, 
                    EMBEDDING_DIM, 
                    mask_zero=True, 
                    name="title_emb",
                    # input_shape=(self.sku_count,)
                ),
                tf.keras.layers.GlobalAveragePooling1D(
                    name="title_flatten",
                    # input_shape=(self.sku_count+1,EMBEDDING_DIM,)
                )
            ], 
            name="title_embedding"
        )
        self.description_embedding = tf.keras.Sequential(
            [
                self.description_vectorizor,
                tf.keras.layers.Embedding(
                    MAX_TOKENS+1, 
                    EMBEDDING_DIM, 
                    mask_zero=True, 
                    name="desc_emb",
                    # input_shape=(self.sku_count,)
                ),
                tf.keras.layers.GlobalAveragePooling1D(
                    name="desc_flatten",
                    # input_shape=(self.sku_count+1,EMBEDDING_DIM,)
                )
            ], 
            name="description_embedding"
        )
        self.category_embedding = tf.keras.Sequential(
           [
               self.category_vectorizor,
               tf.keras.layers.Embedding(
                   MAX_TOKENS+1, 
                   EMBEDDING_DIM, 
                   mask_zero=True, 
                   name="category_emb",
                   # input_shape=(MAX_TOKENS,)
               ),
               tf.keras.layers.GlobalAveragePooling1D(
                   name="category_flatten",
                   # input_shape=(MAX_TOKENS+1,EMBEDDING_DIM,)
               )
           ], 
           name="category_embedding"
        )
        
        
        # Then construct the layers.
        self.dense_layers = tf.keras.Sequential(name="dense_layers_product")
        
        # Adding weight initialzier
        initializer = tf.keras.initializers.GlorotUniform(seed=SEED)
        # Use the ReLU activation for all but the last layer.
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(tf.keras.layers.Dense(
                layer_size,
                activation="relu",
                kernel_initializer=initializer
            ))
            if DROPOUT:
                self.dense_layers.add(tf.keras.layers.Dropout(
                    DROPOUT_RATE
                ))
        # No activation for the last layer
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(tf.keras.layers.Dense(
                layer_size,
                kernel_initializer=initializer
            ))
        ### ADDING L2 NORM AT THE END
        self.dense_layers.add(tf.keras.layers.Lambda(
            lambda x: tf.nn.l2_normalize(
                x,
                1,
                epsilon=1e-12,
                name="normalize_dense"
            )
        ))
    def call(self, data):
        all_embs = tf.concat(
            [
                self.description_embedding(data['description']), 
                self.sku_embedding(data['productId']), 
                self.category_embedding(data['categories']),  
                self.title_embedding(data['title']),  
                # tf.reshape(self.price_normalization(data["price"]), (-1, 1)),
            ], axis=1)
        return self.dense_layers(all_embs)  

## User Model Class and Data

In [27]:
# customer_data_sql = """
# CREATE OR REPLACE TABLE `css_retail.users_train` AS
# SELECT
#     SAFE_CAST(id as STRING) AS userId,
#     SAFE_CAST(age as FLOAT64) AS age,
#     gender,
#     latitude,
#     longitude,
#     zip,
#     traffic_source,
#     SAFE_CAST(TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), created_at, DAY) AS FLOAT64) AS customer_lifetime_days
# FROM `css_retail.customers` AS customers
# """
# _ = client.query(customer_data_sql)
# _.result()

In [28]:
class UserModel(tf.keras.Model):
    def __init__(self, adapt_data):
        super().__init__()
        
        logging.info('Preprocessing Running - user model')
        
        #preprocess stuff
        self.user_lookup = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS, input_shape=())
        logging.info('User Lookup Complete - user model')
        
        self.gender_vocab = tf.constant(['Female', 'Male'], name="gender vocab")
        self.traffic_vocab = tf.constant(['Email', 'Search', 'Display', 'Organic', 'Facebook'], name="traffic vocab")
        logging.info('Lifetime Buckets Complete - user model')
        self.traffic_source_lookup = tf.keras.layers.TextVectorization(max_tokens=10, input_shape=())
        self.zip_lookup = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS, input_shape=())
        self.gender_lookup = tf.keras.layers.TextVectorization(max_tokens=5, input_shape=())
        
        logging.info('Adapts Running - user model')
        #adapt stuff
        # self.gender_lookup.adapt(adapt_data.map(lambda x: x['gender']))
        self.user_lookup.adapt(adapt_data.map(lambda x: x['userId']))
        self.zip_lookup.adapt(adapt_data.map(lambda x: x['zip']))
        # self.traffic_source_lookup.adapt(adapt_data.map(lambda x: x['traffic_source']))
        self.lt_disc = tf.keras.layers.Discretization(
                num_bins=N_BINS,
                name="lifetime_disc",
                # input_shape=()
            )
        self.lt_disc.adapt(adapt_data.map(lambda x: x['customer_lifetime_days']))
        
        self.age_disc = tf.keras.layers.Discretization(
                num_bins=N_BINS,
                name='age_disc',
                # input_shape=()
            )
        self.age_disc.adapt(adapt_data.map(lambda x: x['age']))
        
        #embed stuff
        self.user_embedding = tf.keras.Sequential([
            self.user_lookup,
            tf.keras.layers.Embedding(
                MAX_TOKENS+1,
                EMBEDDING_DIM,
                mask_zero=True,
                name="user_emb",)
                # input_shape=()),
        ,
            tf.keras.layers.GlobalAveragePooling1D(
                name="user_flat",
                #input_shape=(MAX_TOKENS+1,EMBEDDING_DIM,)
            )
        ], name="user_embedding")
        self.age_embedding = tf.keras.Sequential([
            self.age_disc,
            tf.keras.layers.Embedding(
                N_BINS+1,
                EMBEDDING_DIM,
                # input_shape=(N_BINS,)
            )
        ])
        #     ,
        #     tf.keras.layers.GlobalAveragePooling1D(
        #         name="age_flat",
        #         input_shape=(N_BINS+1 ,EMBEDDING_DIM,)
        #     )
        # ], name="age_embedding")
        
        self.lifetime_embedding = tf.keras.Sequential([
            self.lt_disc,
            tf.keras.layers.Embedding(
                N_BINS+1,
                EMBEDDING_DIM,
                # input_shape=(N_BINS,)
            )])
        #     ,
        #     tf.keras.layers.GlobalAveragePooling1D(
        #         name="lt_flat",
        #         input_shape=(N_BINS+1,EMBEDDING_DIM,)
        #     )
        # ], name="customer_lifetime_embedding")
        
        # self.traffic_source_embedding = tf.keras.Sequential([
        #     self.traffic_source_lookup,
        #     tf.keras.layers.Embedding(
        #         11,
        #         EMBEDDING_DIM,
        #         mask_zero=True,
        #         name="traffic_source_emb",
        #         # input_shape=()
        #     ),
        #     tf.keras.layers.GlobalAveragePooling1D(
        #         name="traffic_source_flat",
        #         # input_shape=(N_BINS+1,EMBEDDING_DIM,)
        #     )
        # ], name="traffic_source_embedding")
        
        self.zip_embedding = tf.keras.Sequential([
            self.zip_lookup,
            tf.keras.layers.Embedding(
                MAX_TOKENS+1,
                EMBEDDING_DIM,
                mask_zero=True,
                name="zip_emb",
                # input_shape=()
            ),
            tf.keras.layers.GlobalAveragePooling1D(name="zip_flat"
                                                  # input_shape=(MAX_TOKENS+1,EMBEDDING_DIM,)
                                                  )
        ], name="zip_embedding")
        
        self.gender_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=self.gender_vocab,
                mask_token=None,
                name="gender_lookup",
                output_mode='count',
            input_shape=(1,)
            )
        ], name="gender_emb")
        
        self.traffic_source_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=self.traffic_vocab,
                mask_token=None,
                name="traffic_source_lookup",
                output_mode='count',
            input_shape=(1,)
            )
        ], name="traffic_source_emb")
        
#         self.gender_embedding = tf.keras.Sequential([
#             self.gender_lookup,
#             tf.keras.layers.Embedding(
#                 6,
#                 EMBEDDING_DIM,
#                 mask_zero=True,
#                 name="gender_emb",
#                 # input_shape=()
#             ),
#             tf.keras.layers.GlobalAveragePooling1D(
#                 name="gender_flat",
#                 # input_shape=(5,EMBEDDING_DIM,)
                
#             )
#         ], name="gender_embedding")
           
    def call(self, data):
        all_embs = tf.concat(
            [
                self.user_embedding(data['userId']),
                self.age_embedding(data['age']),
                self.lifetime_embedding(data['customer_lifetime_days']),
                self.traffic_source_embedding(data['traffic_source']),
                self.zip_embedding(data['zip']),
                self.gender_embedding(data['gender'])], axis=1)
        
        return all_embs 

    


## Event Model - Bringing together the data

In [29]:
# purchase_data_sql = """
# CREATE OR REPLACE TABLE `css_retail.train` AS
# WITH inner_q AS (
#     SELECT
#         SAFE_CAST(userInfo.userId AS STRING) AS userId,
#         SAFE_CAST(eventTime AS TIMESTAMP) AS eventTime,
#         productEventDetail.cartId,
#         productEventDetail.purchaseTransaction.revenue,
#         SAFE_CAST(products.id as string) productId,
#         products.quantity,
#         products.displayPrice AS price
#     FROM `css_retail.purchase_complete` AS purchase,
#     UNNEST(productEventDetail.productDetails) AS products
# ) SELECT
#     user.* ,
#     inner_q.* EXCEPT (eventTime, userId, cartId, price, revenue),
#     UNIX_MILLIS(eventTime) AS eventTime,
#     SAFE_CAST(EXTRACT(HOUR FROM eventTime) AS STRING) AS hour,
#     SAFE_CAST(EXTRACT(DAY FROM eventTime) AS STRING) AS day,
#     SAFE_CAST(EXTRACT(MONTH FROM eventTime) AS STRING) AS month,
#     SAFE_CAST(EXTRACT(DAYOFWEEK FROM eventTime) AS STRING) AS dow,
    
#     product.* EXCEPT (productId)
# FROM inner_q JOIN `css_retail.users_train` as user ON inner_q.userId = user.userId
# JOIN `css_retail.product_train` as product ON inner_q.productId = product.productId
# """
# _ = client.query(purchase_data_sql)
# _.result()

In [30]:
class EventModel(tf.keras.Model):
    def __init__(self, layer_sizes, adapt_data):
        super().__init__()
        logging.info('Preprocessing Running - query model')
        ## Bring in user data
        logging.info('Obtaining user embeddings')
        self.user_embs = UserModel(adapt_data) #for user data, run embeddings
        logging.info('finished with user embeddings')
        # self.product_embs = 
        ### preprocess stuff
        
        self.month_vocab = tf.constant([str(i) for i in range(1,12)], name="month_vocab")
        self.day_vocab = tf.constant([str(i) for i in range(1,31)], name="day_vocab")
        self.dow_vocab = tf.constant([str(i) for i in range(1,7)], name="dow_vocab")
        self.hour_vocab = tf.constant([str(i) for i in range(0,24)], name="hour_vocab")
        
        logging.info('finished vocabs - EventModel')
        
        # self.eventtime_normalization = tf.keras.layers.Normalization()
        
        ### adapt stuff
        # self.eventtime_normalization.adapt(adapt_data.map(lambda x: x['eventTime']))
        logging.info('Adapts Complete - EventModel')

        ## embed stuff
        
        self.month_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=self.month_vocab,
                mask_token=None,
                name="month_lookup",
                output_mode='count',
            input_shape=(1,)
            )
        ], name="month_emb")
        
        self.hour_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=self.hour_vocab,
                mask_token=None,
                name="hour_lookup",
                output_mode='count',
            input_shape=(1,)
            )
        ], name="hour_emb")
        
        self.day_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=self.day_vocab,
                mask_token=None,
                name="day_lookup",
                output_mode='count',
            input_shape=(1,)
            )
        ], name="day_emb")
        
        self.dow_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=self.dow_vocab,
                mask_token=None,
                name="dow_lookup",
                output_mode='count',
            input_shape=(1,)
            )
        ], name="dow_emb")
        logging.info('Adapts complete - query model')
        # Then construct the layers.
        self.dense_layers = tf.keras.Sequential(name="dense_layers_query")

        initializer = tf.keras.initializers.GlorotUniform(seed=SEED)
        # Use the ReLU activation for all but the last layer.
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(tf.keras.layers.Dense(
                layer_size,
                activation="relu",
                kernel_initializer=initializer
            ))
            if DROPOUT:
                self.dense_layers.add(tf.keras.layers.Dropout(
                    DROPOUT_RATE
                ))
        # No activation for the last layer
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(tf.keras.layers.Dense(
                layer_size,
                kernel_initializer=initializer
            ))
        ### ADDING L2 NORM AT THE END
        self.dense_layers.add(tf.keras.layers.Lambda(
            lambda x: tf.nn.l2_normalize(
                x,
                1,
                epsilon=1e-12,
                name="normalize_dense"
            )
        ))

# tf.reshape(self.price_normalization(data["price"]), (-1, 1))
    def call(self, data):
        all_embs = tf.concat(
            [
                self.month_embedding(data['month']), 
                self.dow_embedding(data['dow']),
                self.day_embedding(data['day']), 
                self.hour_embedding(data['hour']),
                self.user_embs(data),
                # tf.reshape(self.eventtime_normalization(data['eventTime']), (-1, 1))
                           ]
              , axis=1)

        return self.dense_layers(all_embs)

## Two-Tower Model

In [31]:
class TheTwoTowers(tfrs.models.Model):
    def __init__(self, layer_sizes, query_adapt_data, cat_adapt_data):
        super().__init__()
        self.cat_adapt_data = cat_adapt_data.cache()
        self.candidate_model = ProductModel(layer_sizes, self.cat_adapt_data)
        self.query_model = EventModel(layer_sizes, query_adapt_data)
        
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=self.cat_adapt_data.map(self.candidate_model),
            )
        )

    def compute_loss(self, data, training=False):
        query_embeddings = self.query_model(data)
        product_embeddings = self.candidate_model(data)

        return self.task(
            query_embeddings,
            product_embeddings,
            compute_metrics=not training
        )#### turn off metrics to save time on training


In [32]:
def get_arch_from_string(arch_string):
    q = arch_string.replace(']', '')
    q = q.replace('[', '')
    q = q.replace(" ", "")
    return [int(x) for x in q.split(',')]

## Training Script

In [33]:
client = BigQueryClient()


candidates_adapt = read_dataset_prod(client, None, BATCH_SIZE)
candidates_adapt = candidates_adapt.map(lambda x: x).cache()




parsed_dataset_adapt = read_dataset(client, None, BATCH_SIZE)

logging.info('Setting model adapts and compiling the model')

model = TheTwoTowers( get_arch_from_string(ARCH), query_adapt_data=parsed_dataset_adapt, cat_adapt_data=read_dataset_prod(BigQueryClient(), None, 2048))
model.compile(optimizer=tf.keras.optimizers.Adagrad(LR))
logging.info('Adapts finish - training next')
tf.random.set_seed(SEED)
#time split at 95th quantile of `eventTime` below
tensorboard_cb = tf.keras.callbacks.TensorBoard(JOB_DIR, histogram_freq=0)

# test_cache = test.cache()

# tf.data.experimental.save(test, 'test_cached')


INFO:absl:Setting model adapts and compiling the model
INFO:absl:Preprocessing Running - product model


2021-11-15 17:53:31.349025: E tensorflow/core/framework/dataset.cc:552] Unimplemented: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2021-11-15 17:53:31.349067: E tensorflow/core/framework/dataset.cc:556] Unimplemented: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.


INFO:absl:Lookups Complete - product model
INFO:absl:Adapts Running - product model
INFO:absl:Starting Emb Layers - product model
INFO:absl:Preprocessing Running - query model
INFO:absl:Obtaining user embeddings
INFO:absl:Preprocessing Running - user model
INFO:absl:User Lookup Complete - user model
INFO:absl:Lifetime Buckets Complete - user model
INFO:absl:Adapts Running - user model


2021-11-15 17:53:33.304759: E tensorflow/core/framework/dataset.cc:552] Unimplemented: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2021-11-15 17:53:33.304810: E tensorflow/core/framework/dataset.cc:556] Unimplemented: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2021-11-15 17:53:45.296772: E tensorflow/core/framework/dataset.cc:552] Unimplemented: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2021-11-15 17:53:45.296819: E tensorflow/core/framework/dataset.cc:556] Unimplemented: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2021-11-15 17:53:56.951182: E tensorflow/core/framework/dataset.cc:552] Unimplemented: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the 

INFO:absl:finished with user embeddings
INFO:absl:finished vocabs - EventModel
INFO:absl:Adapts Complete - EventModel
INFO:absl:Adapts complete - query model
INFO:absl:Adapts finish - training next


2021-11-15 17:54:21.051075: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-15 17:54:21.051113: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-15 17:54:21.175842: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-11-15 17:54:21.176008: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed


In [ ]:
# test = read_dataset(BigQueryClient(), 'eventTime > 1637994570450', BATCH_SIZE)
train = read_dataset(BigQueryClient(), 'eventTime <= 1629906782450', BATCH_SIZE)
test_client = BigQueryClient()
test = read_dataset(test_client, 'eventTime > 1629906782450', BATCH_SIZE)
test = test.cache()

layer_history = model.fit(
    train,
    validation_data=test,
    validation_freq=3,
    callbacks=[tensorboard_cb],
    epochs=NUM_EPOCHS,
    verbose=0)

2021-11-15 17:54:24.109605: E tensorflow/core/framework/dataset.cc:552] Unimplemented: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2021-11-15 17:54:24.109665: E tensorflow/core/framework/dataset.cc:556] Unimplemented: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2021-11-15 17:54:24.534688: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-15 17:54:24.534735: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-15 17:54:25.059348: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-15 17:54:25.060413: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2021-11-15 17:54:25.192897: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 620 c

In [ ]:
model.evaluate(test, return_dict=True)

## Plot performance

In [ ]:
model.weights

In [ ]:
# shuffled = read_bigquery('train', EVENT_USER_SCHEMA, UNUSED_COLUMNS_U).shuffle(10000).batch(BATCH_SIZE)


# train_records
# train = shuffled.skip(310_000).cache()


model.evaluate(tf.data.experimental.load('test_cached'))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(NUM_EPOCHS), layer_history.history["loss"], label=ARCH)
plt.title("Loss vs epoch")
plt.xlabel("epoch")
plt.ylabel("Loss Metric");
plt.legend()

## Save the query/event model

In [ ]:
# Export the candidate model.
import tempfile
candidates = read_bigquery('product_train', PRODUCT_SCHEMA, UNUSED_COLUMNS_P).shuffle(200)
scann_index = tfrs.layers.factorized_top_k.ScaNN(model.candidate_model, num_reordering_candidates=1000)

with tempfile.TemporaryDirectory() as tmp:
    path = os.path.join(tmp, "model")

    # Save the index.
    tf.saved_model.save(scann_index, JOB_DIR,  options=tf.saved_model.SaveOptions(namespace_whitelist=["Scann"]))

    # Load it back; can also be done in TensorFlow Serving.
    loaded = tf.saved_model.load(JOB_DIR)

In [ ]:
# Export the query model.
import tempfile
candidates = read_bigquery('product_train', PRODUCT_SCHEMA, UNUSED_COLUMNS_P).shuffle(200)
scann_index = tfrs.layers.factorized_top_k.ScaNN(model.query_model, num_reordering_candidates=1000)

with tempfile.TemporaryDirectory() as tmp:
    path = os.path.join(tmp, "model")

    # Save the index.
    tf.saved_model.save(scann_index, JOB_DIR,  options=tf.saved_model.SaveOptions(namespace_whitelist=["Scann"]))

    # Load it back; can also be done in TensorFlow Serving.
    loaded = tf.saved_model.load(JOB_DIR)